# 데이터 준비하기 : 조인, 병합, 변형

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

저번 해커톤을 생각해 보면, csv 파일이 무려 8개였습니다.<br>이렇듯 여러 파일이나 데이터 베이스는 분산되어 있는 경우가 있는데 이런 데이터를 합치고 재배열하는 방법을 살펴보자!

## 계층적 색인 : Hierarchical Indexing
- 축에 대해 둘 이상의 다중 색인 **단계**를 지정

In [9]:
# 예시 : 리스트를 색인으로 하는 Series
single_index_data = pd.Series(np.random.randn(9),
                              index=['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'])

# 예시 : 중첩 리스트를 색인으로 하는 Series
multi_index_data = pd.Series(np.random.randn(9),
                             index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                                    [1, 2, 3, 1, 3, 1, 2, 2, 3]])
display(single_index_data)
display(multi_index_data)

a   -1.265934
a    0.119827
a   -1.063512
b    0.332883
b   -2.359419
c   -0.199543
c   -1.541996
d   -0.970736
d   -1.307030
dtype: float64

a  1    0.286350
   2    0.377984
   3   -0.753887
b  1    0.331286
   3    1.349742
c  1    0.069877
   2    0.246674
d  2   -0.011862
   3    1.004812
dtype: float64

예시에서 중첩 리스트를 이용하여 생성된 Series의 색인(index)를 출력하면 `MultiIndex` 꼴임을 확인할 수 있다.<br>

In [12]:
display(single_index_data.index)
display(multi_index_data.index)

Index(['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], dtype='object')

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

- 상위 단계의 색인을 이용하여 하위 계층에 접근할 수 있다.
    - 배열에서 상위 차원의 색인을 통하여 하위 차원의 값을 찾아가는 것처럼...!
    - 예를들어 shape가 (2,3)인 ***배열A***에서 (1,1) 위치의 원소에 접근하기 위해서는 A[1][1]로 접근한다.
    - 예를들어 shape가 (2,3)인 ***배열A***에서 0번째 row의 모든 원소에 접근하기 위해서는 A[0]로 접근한다.
- 계층적으로 색인된 객체는 데이터의 부분집합을 **부분적 색인**으로 접근하는 것이 가능하다.
    - 다른말을 사용하였지만, 위와 같은 이야기로 슬라이싱이 가능하다는 말이다.

In [23]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.670216
   2    0.852965
   3   -0.955869
b  1   -0.023493
   3   -2.304234
c  1   -0.652469
   2   -1.218302
d  2   -1.332610
   3    1.074623
dtype: float64

In [21]:
data['b']

1   -0.713544
3   -0.831154
dtype: float64

In [5]:
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

- 하위 계층의 객체를 선택하는 것도 가능 👉 loc, iloc 사용가능

In [24]:
data.loc[:, 2]

a    0.852965
c   -1.218302
d   -1.332610
dtype: float64

- unstack
    - MultiIndex에서 하나의 index를 column으로 만들어 데이터를 새롭게 배열한다.
    - index의 특정 계층을 column의 최하위 계층으로 올린다.
    - <u>높게 쌓여 있는 것을 옆</u>으로 퍼뜨린다.
    - ![unstack_parms](https://i.ibb.co/HYSP5qJ/image.png)

In [52]:
display(data)
display(data.index.levels[-1])

a  1    0.670216
   2    0.852965
   3   -0.955869
b  1   -0.023493
   3   -2.304234
c  1   -0.652469
   2   -1.218302
d  2   -1.332610
   3    1.074623
dtype: float64

Int64Index([1, 2, 3], dtype='int64')

In [37]:
data.unstack()

,1,2,3
a,0.670216,0.852965,-0.955869
b,-0.023493,NaN,-2.304234
c,-0.652469,-1.218302,NaN
d,NaN,-1.332610,1.074623


- stack
    - DataFrame에서 하나의 column를 index으로 만들어 데이터를 새롭게 배열한다.
    - column의 특정 계층을 index의 최하위 계층으로 내린다.
    - <u>퍼뜨려진 것을 높게 쌓는다</u>.
    - ![unstack_parms](https://i.ibb.co/NTg0B4v/image.png)

In [38]:
data.unstack().stack()

a  1    0.670216
   2    0.852965
   3   -0.955869
b  1   -0.023493
   3   -2.304234
c  1   -0.652469
   2   -1.218302
d  2   -1.332610
   3    1.074623
dtype: float64

- `stack - unstack 어디에 쓸까?`

In [88]:
air_visit_data = pd.read_csv('./data/air_visit_data.csv', 
                         parse_dates=['visit_date'])
display(air_visit_data.head())
len(air_visit_data)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


252108

In [31]:
air_visit_data_multi = air_visit_data.groupby(['air_store_id', 'visit_date']).sum()
air_visit_data_multi.head(5)

visitors
air_store_id         visit_date          
air_00a91d42b08b08d9 2016-07-01        35
                     2016-07-02         9
                     2016-07-04        20
                     2016-07-05        25
                     2016-07-06        29

In [35]:
air_visit_data_multi.unstack().fillna(0).astype('int')

visitors                                              \
visit_date           2016-01-01 2016-01-02 2016-01-03 2016-01-04 2016-01-05   
air_store_id                                                                  
air_00a91d42b08b08d9          0          0          0          0          0   
air_0164b9927d20bcc3          0          0          0          0          0   
air_0241aa3964b7f861          0          0         10          9         17   
air_0328696196e46f18          0          0          0          0          0   
air_034a3d5b40d5b1b1          0          0          0          0          0   
...                         ...        ...        ...        ...        ...   
air_fea5dc9594450608          0          0          0          0          0   
air_fee8dcf4d619598e          0          0          0          0          0   
air_fef9ccb3ba0da2f7          0          0          0          0          0   
air_ffcc2d5087e1b476          0          0          0          0          0   
air_fff68b929994bfbd          0          0          0          0          0   

                                                                             \
visit_date           2016-01-06 2016-01-07 2016-01-08 2016-01-09 2016-01-10   
air_store_id                                                                  
air_00a91d42b08b08d9          0          0          0          0          0   
air_0164b9927d20bcc3          0          0          0          0          0   
air_0241aa3964b7f861         10          0          5          8         16   
air_0328696196e46f18          0          0          0          0          0   
air_034a3d5b40d5b1b1          0          0          0          0          0   
...                         ...        ...        ...        ...        ...   
air_fea5dc9594450608          0          0          0          0          0   
air_fee8dcf4d619598e          0          0          0          0          0   
air_fef9ccb3ba0da2f7          0          0          0          0          0   
air_ffcc2d5087e1b476          0          0          0          0          0   
air_fff68b929994bfbd          0          0          0          0          0   

                      ...                                              \
visit_date            ... 2017-04-13 2017-04-14 2017-04-15 2017-04-16   
air_store_id          ...                                               
air_00a91d42b08b08d9  ...         34         39          0          0   
air_0164b9927d20bcc3  ...         13          7          1          0   
air_0241aa3964b7f861  ...          0          4         15         10   
air_0328696196e46f18  ...          0          9          4          3   
air_034a3d5b40d5b1b1  ...         22         18         31         39   
...                   ...        ...        ...        ...        ...   
air_fea5dc9594450608  ...         17         20         20         15   
air_fee8dcf4d619598e  ...         11         37         47         32   
air_fef9ccb3ba0da2f7  ...          6          1         23          9   
air_ffcc2d5087e1b476  ...         16         37         25          0   
air_fff68b929994bfbd  ...          1          5          7          7   

                                                                             \
visit_date           2017-04-17 2017-04-18 2017-04-19 2017-04-20 2017-04-21   
air_store_id                                                                  
air_00a91d42b08b08d9         19         35         17         38         55   
air_0164b9927d20bcc3          2          1          8          1         26   
air_0241aa3964b7f861         12         19          8          0          3   
air_0328696196e46f18          3          0         24          0         19   
air_034a3d5b40d5b1b1         25         20         31         12         37   
...                         ...        ...        ...        ...        ...   
air_fea5dc9594450608          9         16         17         23         28

- DataFrame에서는 두 축 모두 계층적 색인을 가질 수 있다.

In [55]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [57]:
# 계층적 색인의 각 단계는 이름을 가질 수 있다.
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [58]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

- MultiIndex는 따로 생성한 다음 재사용 가능하다.
```python
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                          names=['state', 'color'])
```

### 계층의 순서를 바꾸고 정렬하기

#### 계층 순서 바꾸기 : `swaplevel`
**`DataFrame.swaplevel(i=- 2, j=- 1, axis=0)`**
> Swap levels i and j in a MultiIndex.<br>
> Default is to swap the two innermost levels of the index.

**Parameters**
- i, j : int or str, Levels of the indices to be swapped. Can pass level name as string.
- axis : {0 or ‘index’, 1 or ‘columns’}, default 0
    - The axis to swap levels on.
    - 0 or ‘index’ for row-wise, 1 or ‘columns’ for column-wise.

**Returns**
- DataFrame : DataFrame with levels swapped in MultiIndex.

In [63]:
display(frame)
print(">> swaplevel(key1, key2)")
display(frame.swaplevel('key1', 'key2'))

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

>> swaplevel(key1, key2)


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

#### 계층 정렬하기: `sort_index`
![image.png](https://i.ibb.co/tBPxw2d/image.png)

In [72]:
_frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [2, 1, 2, 1]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
display(_frame)
display(_frame.sort_index(level=0, sort_remaining=False))
display(_frame.sort_index(level=0, sort_remaining=True))
display(_frame.sort_index(level=0, ignore_index=True, sort_remaining=True))

Ohio     Colorado
    Green Red    Green
a 2     0   1        2
  1     3   4        5
b 2     6   7        8
  1     9  10       11

Ohio     Colorado
    Green Red    Green
a 2     0   1        2
  1     3   4        5
b 2     6   7        8
  1     9  10       11

Ohio     Colorado
    Green Red    Green
a 1     3   4        5
  2     0   1        2
b 1     9  10       11
  2     6   7        8

Ohio     Colorado
  Green Red    Green
0     3   4        5
1     0   1        2
2     9  10       11
3     6   7        8

In [89]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [90]:
# swaplevel을 이용해서 계층
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

- DataFrame과 Series의 많은 기술 통계와 요약 통계는 level 옵션을 가지고 있다.
    - 기술 통계 : count, sum, mean, median, mode, std, min, max, abs, prod, cumsum, cumprod, describe
    - 요약 통계 : describe

In [93]:
display(frame)
print(">> sum(level=key2)")
display(frame.sum(level='key2'))
print(">> sum(level=color)")
display(frame.sum(level='color', axis=1))

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

>> sum(level=key2)


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

>> sum(level=color)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### DataFrame의 컬럼 사용하기

In [94]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


- column을 색인으로 사용하기 : set_index

In [95]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [96]:
# column을 유지하면서 색인으로 사용 : drop=True
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

- 색인을 column으로 사용하기 : reset_index

In [98]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
